# Administrative Descriptive Stats Preparation

After appending travel time information to each populated place in an administrative center we can prepare any number of descriptive stats. Given the quantity of data in question these are best prepared with Dask Dataframes. This notebook separates out the descriptive stats preparations,

In [1]:
import os, sys
from datetime import date

import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import stats

import re

# custom functions
import sensitivity_testing as st

## Setup

### Data prep

Dates

In [2]:
today = date.today().strftime("%y%m%d")

In [3]:
# data_date = '211022'
data_date = '211215'

Directories

In [4]:
geo_dir = r'P:\PAK\GEO'
data_dir = r'../../data'

rast_dir = r'rast_inputs'
vect_in_dir = r'vect_inputs'
vect_out_dir = r'vect_out'

rds_dir = r'roads'
dest_dir = r'destinations'
speed_dir = r'speed'
fric_dir = r'friction'
acc_dir = r'access'
tab_dir = r'tabular'

Projections

In [5]:
# change this to whatever the desired output projection is
DEST_CRS = 'EPSG:32642'

dcrs_int = int(re.findall('[0-9]+',DEST_CRS)[0])
dcrs_int

32642

## Create master files merging admin aggregates and secondary data

There's a lot of primary (modeled) data, secondary data, and spatial data to bring together. Let's do that first

Spatial data

In [35]:
# adm2_geo = gpd.read_file('../../Boundaries/KP_Analysis/KP_Analysis_Focus_Districts.gpkg')
# adm3_geo = gpd.read_file(os.path.join(geo_dir,'Boundaries/KP_Analysis/KP_Analysis_Focus_Tehsils.gpkg')).rename({'ADM1_PCODE':'Adm1_Code','ADM2_PCODE' : 'Adm2_Code','ADM3_PCODE':'Adm3_Code'},axis=1)
adm3_geo = gpd.read_file(os.path.join(geo_dir,'Boundaries/KP_Analysis/KP_Analysis_All_Tehsils.gpkg')).rename({'ADM1_PCODE':'Adm1_Code','ADM2_PCODE' : 'Adm2_Code','ADM3_PCODE':'Adm3_Code'},axis=1)

In [36]:
adm2_ds = adm3_geo.dissolve(by='Adm2_Code')
adm2_geo = adm2_ds.reset_index()[['geometry','ADM1_EN','ADM2_EN','Adm1_Code','Adm2_Code']]
adm3_geo = adm3_geo.drop('Adm2_Code',axis=1)

Tabular data

In [24]:
# access mean

adm2_acc =  pd.read_csv(os.path.join(data_dir,tab_dir,f"processed//adm2_mean_{data_date}.csv"))
adm3_acc =  pd.read_csv(os.path.join(data_dir,tab_dir,f"processed//adm3_mean_{data_date}.csv"))

# # access standard deviation

adm2_sd =  pd.read_csv(os.path.join(data_dir,tab_dir,f"processed//adm2_sd_{data_date}.csv"))
adm3_sd =  pd.read_csv(os.path.join(data_dir,tab_dir,f"processed//adm3_sd_{data_date}.csv"))

# elevation
adm2_elev =  pd.read_csv(os.path.join(data_dir,tab_dir,"processed//adm2_elev.csv"))
adm3_elev =  pd.read_csv(os.path.join(data_dir,tab_dir,"processed//adm3_elev.csv"))

# educational gender ratios
adm3_educ_ratio = pd.read_csv(os.path.join(data_dir,tab_dir,'processed//adm3_educ_gender_ratio.csv')).drop(['Adm2_Code'],axis=1)

# agricultural land usage
adm3_agr = pd.read_csv(os.path.join(data_dir,tab_dir,'processed//adm3_KP_Agricultural_Areas.csv'))

# agricultural production
adm2_agrprod = pd.read_csv(os.path.join(data_dir,tab_dir,'processed//adm2_crop_hazard_d4p_KPK_211015.csv'))

# terrain roughness index
adm3_TRI = pd.read_csv(os.path.join(data_dir,tab_dir,'processed//adm3_TRI.csv'))


In [25]:
# clean up ag data slightly
adm2_agrprod = adm2_agrprod[adm2_agrprod['year'] == 2013].iloc[:,5:].drop(['district_data4pakistan','district_data4pakistan_num'],axis=1)

Unify tabular data

In [26]:
from functools import reduce

adm2_dfs = [adm2_acc,adm2_elev] # agrprod is missing 3 districts and thus returning nulls for them. Took it out for now.
adm3_dfs = [adm3_acc,adm3_elev,adm3_TRI,adm3_agr,adm3_educ_ratio] # 

adm2_tab = reduce(lambda left,right: pd.merge(left,right,on='Adm2_Code'), adm2_dfs)
adm3_tab = reduce(lambda left,right: pd.merge(left,right,how='left',on='Adm3_Code'), adm3_dfs)

In [27]:
adm3_tab

,Adm3_Code,dry_District_HQs_avg_adm3,dry_education_allboys_avg_adm3,dry_education_allgirls_avg_adm3,dry_education_boys_avg_adm3,dry_education_boys_high_avg_adm3,dry_education_boys_middle_avg_adm3,dry_education_boys_primary_avg_adm3,dry_education_girls_avg_adm3,dry_education_girls_high_avg_adm3,...,irrig_sqkm,non_irrig_sqkm,total_agr_sqkm,orch_pct,irrig_pct,non_irrig_pct,total_agr_pct,Adm3_En,Boys_schools,Girls_schools
0,PK20602,2.650377,0.446658,0.724323,0.446658,0.850377,0.896065,0.451359,0.724323,1.190307,...,7.007183,43.932157,54.911789,0.000455,0.000802,0.005031,0.006288,Mastuj,2.402586,9.160957
1,PK20601,1.521887,0.331295,0.575988,0.331295,0.711687,0.815872,0.340370,0.575988,1.008458,...,6.931465,31.564670,42.336770,0.000649,0.001171,0.005335,0.007155,Chitral,2.476649,10.744338
2,PK21201,3.321178,1.055824,1.324615,1.055824,2.707862,1.941663,1.064685,1.324615,3.274572,...,1.589743,28.673123,30.635728,0.000066,0.000283,0.005102,0.005451,Dassu,1.417964,inf
3,PK22206,2.341577,0.397440,0.473922,0.397440,0.627889,0.641474,0.400587,0.473922,1.161092,...,1.395410,22.138573,23.610657,0.000036,0.000663,0.010514,0.011214,Kalam,2.156381,inf
4,PK23003,2.147477,0.473540,0.736336,0.473540,0.799106,0.741702,0.499933,0.736336,1.459440,...,0.907668,16.886898,18.000187,0.000268,0.001181,0.021972,0.023420,Kalkot,13.805527,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,PK23808,0.779918,0.459722,0.486790,0.459722,0.614233,0.601355,0.467104,0.486790,0.677857,...,3.268404,54.088002,58.332932,0.000437,0.001462,0.024187,0.026085,NaN,NaN,NaN
112,PK20703,1.151925,0.147241,0.342242,0.147302,0.507031,0.562369,0.155795,0.343041,0.778638,...,67.146160,717.361577,796.755828,0.010939,0.059968,0.640669,0.711576,Kulachi,5.683960,56.793002
113,PK23807,3.586709,1.803391,1.786410,1.803391,2.803082,1.918273,1.817956,1.786410,3.168727,...,0.915049,34.596888,35.544239,0.000062,0.001746,0.066024,0.067832,NaN,NaN,NaN
114,PK20701,2.340478,0.174527,0.417384,0.174527,1.183932,0.799227,0.178196,0.417384,1.376033,...,30.244979,1154.694670,1209.756072,0.014533,0.017713,0.676233,0.708479,Daraban,7.875748,18.156053


In [28]:
adm3_tab = pd.merge(adm3_tab,adm2_agrprod,how='left',on='Adm2_Code')

In [29]:
adm3_tab.columns

Index(['Adm3_Code', 'dry_District_HQs_avg_adm3',
       'dry_education_allboys_avg_adm3', 'dry_education_allgirls_avg_adm3',
       'dry_education_boys_avg_adm3', 'dry_education_boys_high_avg_adm3',
       'dry_education_boys_middle_avg_adm3',
       'dry_education_boys_primary_avg_adm3', 'dry_education_girls_avg_adm3',
       'dry_education_girls_high_avg_adm3',
       ...
       'sp3avg_by_s_cat56', 'sp3avg_by_s_cat5', 'sp3avg_by_s_cat6',
       'mean_povrate', 'pov_quintile', 'pov_quintile_2014',
       'flood_1in10_mean_tercile', 'flood_1in10_max_tercile',
       'flood_1in50_mean_tercile', 'flood_1in50_max_tercile'],
      dtype='object', length=480)

#### Compute agricultural totals

In [31]:
yield_cols = ['yield_wt','yield_ba','yield_by','yield_cn','yield_gs','yield_jr','yield_re','yield_st','yield_se']
share_irriP_cols = ['share_irriP_wt','share_irriP_ba','share_irriP_by','share_irriP_cn','share_irriP_gs','share_irriP_jr','share_irriP_re','share_irriP_st','share_irriP_se']
share_irriA_cols = ['share_irriA_wt','share_irriA_ba','share_irriA_by','share_irriA_cn','share_irriA_gs','share_irriA_jr','share_irriA_re','share_irriA_st','share_irriA_se']

In [32]:
adm3_tab['yield_to'] = np.sum(adm3_tab[yield_cols],axis=1)
adm3_tab['share_irriP_to'] = np.sum(adm3_tab[share_irriP_cols],axis=1)
adm3_tab['share_irriA_to'] = np.sum(adm3_tab[share_irriA_cols],axis=1)

In [33]:
adm3_tab[['yield_to','share_irriP_to','share_irriA_to']]

,yield_to,share_irriP_to,share_irriA_to
0,5.010840,1.977012,2.963415
1,5.010840,1.977012,2.963415
2,2.857143,0.653846,0.571429
3,4.204707,0.343217,0.594551
4,3.869048,0.565217,0.444444
...,...,...,...
111,1.041667,0.600000,0.583333
112,57.634802,3.066572,3.039518
113,1.041667,0.600000,0.583333
114,57.634802,3.066572,3.039518


#### Consolidate final datasets

Unify spatial and tabular data

In [37]:
adm2_final = pd.merge(adm2_geo,adm2_tab,how='left',on='Adm2_Code')
adm3_final = pd.merge(adm3_geo,adm3_tab,how='left',on='Adm3_Code')

In [95]:
adm3_final

,Shape_Leng,Shape_Area,ADM3_EN,Adm3_Code,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM2_EN,ADM1_EN,Adm1_Code,...,sp3avg_by_s_cat56,sp3avg_by_s_cat5,sp3avg_by_s_cat6,mean_povrate,pov_quintile,pov_quintile_2014,flood_1in10_mean_tercile,flood_1in10_max_tercile,flood_1in50_mean_tercile,flood_1in50_max_tercile
0,2.187495,0.171564,Abbottabad,PK20101,None,None,None,Abbottabad,Khyber Pakhtunkhwa,PK2,...,0.0,0.0,0.0,NaN,NaN,1.0,3.0,2.0,3.0,2.0
1,1.581975,0.080696,Alai,PK20301,None,None,None,Batagram,Khyber Pakhtunkhwa,PK2,...,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0,3.0,2.0
2,1.189227,0.056426,Alpuri,PK22001,None,None,None,Shangla,Khyber Pakhtunkhwa,PK2,...,0.0,0.0,0.0,NaN,NaN,3.0,3.0,2.0,3.0,2.0
3,0.818266,0.022875,Ambar Utman Khel,PK23501,None,None,None,Mohmand,Khyber Pakhtunkhwa,PK2,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.0,2.0
4,0.948441,0.031578,Babuzai,PK22201,None,None,None,Swat,Khyber Pakhtunkhwa,PK2,...,0.0,0.0,0.0,NaN,NaN,2.0,3.0,2.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.671976,0.016740,Utman Khel Tehsil,PK23207,None,Utman Khel,None,Bajaur,Khyber Pakhtunkhwa,PK2,...,0.0,0.0,0.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0
112,3.496818,0.213873,Wana,PK23808,None,None,None,South Waziristan,Khyber Pakhtunkhwa,PK2,...,0.0,0.0,0.0,NaN,NaN,NaN,2.0,2.0,1.0,2.0
113,1.075965,0.064361,Wari,PK23004,None,None,None,Upper Dir,Khyber Pakhtunkhwa,PK2,...,0.0,0.0,0.0,NaN,NaN,3.0,3.0,2.0,3.0,2.0
114,1.995740,0.079313,Wazir,PK23901,None,None,None,FR Bannu,Khyber Pakhtunkhwa,PK2,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0


#### Export

Export these interim products

In [39]:
# # all KP
adm2_tab.to_csv(os.path.join(data_dir,tab_dir,f"final//adm2_all_KP_raw_access_aggregates_{today}.csv"),index=False)
adm3_tab.to_csv(os.path.join(data_dir,tab_dir,f"final//adm3_all_KP_raw_access_aggregates_{today}.csv"),index=False)

In [ ]:
# all KP -- geo
# adm2_final.to_file(os.path.join(data_dir,acc_dir,f"vector/adm2_all_KP_access_raw_aggregates_{today}.gpkg"),driver="GPKG")
# adm3_final.to_file(os.path.join(data_dir,acc_dir,f"vector/adm3_all_KP_access_raw_aggregates_{today}.gpkg"),driver="GPKG")

## Consolidate access variables into master columns per category, and thence an overall dataset

This step slims down the administrative dataset into only the columns we anticipate using for our analysis (ie. we drop children's walking speeds to Peshawar).</br></br>
Seasonal figures are weighted by the season's length in that tehsil and then merged into a master value per column (e.g. health_primary) and then again merged, with weighting, into a sectoral master (e.g. health_idx). These are merged into a final inaccessibility index value.</br></br>We preserve the intermediate columns in this process to enable sensitivity testing of the importance of the weights used.

#### Define admin level of analysis

In [178]:
# adm_level = 'adm2'
adm_level = 'adm3'

In [179]:
if adm_level == 'adm2':
    df = adm2_final.copy()
    adm_cols = ('ADM2_EN','Adm2_Code')
elif adm_level == 'adm3':
    df = adm3_final.copy()
    adm_cols = ('ADM2_EN','Adm2_Code','ADM3_EN','Adm3_Code')

#### Load in data

Spatial data

In [180]:
# adm2_geo = gpd.read_file('../../Boundaries/KP_Analysis/KP_Analysis_Focus_Districts.gpkg')
# adm3_geo = gpd.read_file(os.path.join(geo_dir,'Boundaries/KP_Analysis/KP_Analysis_Focus_Tehsils.gpkg')).rename({'ADM1_PCODE':'Adm1_Code','ADM2_PCODE' : 'Adm2_Code','ADM3_PCODE':'Adm3_Code'},axis=1)
adm3_geo = gpd.read_file(os.path.join(geo_dir,'Boundaries/KP_Analysis/KP_Analysis_All_Tehsils.gpkg')).rename({'ADM1_PCODE':'Adm1_Code','ADM2_PCODE' : 'Adm2_Code','ADM3_PCODE':'Adm3_Code'},axis=1)

Tabular data

In [181]:
# adm3 = pd.read_csv(os.path.join(data_dir,tab_dir,r"final//adm3_allKP_raw_access_aggregates_{today}.csv"))

In [182]:
df.head(2)

,Shape_Leng,Shape_Area,ADM3_EN,Adm3_Code,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM2_EN,ADM1_EN,Adm1_Code,...,sp3avg_by_s_cat56,sp3avg_by_s_cat5,sp3avg_by_s_cat6,mean_povrate,pov_quintile,pov_quintile_2014,flood_1in10_mean_tercile,flood_1in10_max_tercile,flood_1in50_mean_tercile,flood_1in50_max_tercile
0,2.187495,0.171564,Abbottabad,PK20101,None,None,None,Abbottabad,Khyber Pakhtunkhwa,PK2,...,0.0,0.0,0.0,NaN,NaN,1.0,3.0,2.0,3.0,2.0
1,1.581975,0.080696,Alai,PK20301,None,None,None,Batagram,Khyber Pakhtunkhwa,PK2,...,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0,3.0,2.0


### Prepare for consolidation

Prepare a slimmed down dataframe only containing the access columns for analysis

In [183]:
# first save out the raw data -- we'll join this back in later
unfiltered_acc_cols = list(df.filter(regex='^(childwalk|dry|msn|winter)',axis=1).columns)
raw_acc_data = df[unfiltered_acc_cols].copy().add_suffix('_hrs')

In [184]:
df[unfiltered_acc_cols][:3]

,dry_District_HQs_avg_adm3,dry_education_allboys_avg_adm3,dry_education_allgirls_avg_adm3,dry_education_boys_avg_adm3,dry_education_boys_high_avg_adm3,dry_education_boys_middle_avg_adm3,dry_education_boys_primary_avg_adm3,dry_education_girls_avg_adm3,dry_education_girls_high_avg_adm3,dry_education_girls_middle_avg_adm3,...,childwalk_winter_health_family_avg_adm3,childwalk_winter_health_pharmacies_avg_adm3,childwalk_winter_health_primary_avg_adm3,childwalk_winter_health_private_avg_adm3,childwalk_winter_health_public_avg_adm3,childwalk_winter_health_secondary_avg_adm3,childwalk_winter_health_tertiary_avg_adm3,childwalk_winter_markets_All_avg_adm3,childwalk_winter_markets_Central_avg_adm3,childwalk_winter_Provincial_HQ_avg_adm3
0,0.962309,0.151154,0.182783,0.154368,0.324133,0.390303,0.159064,0.186505,0.458356,0.391861,...,5.626384,1.479962,0.847822,1.690228,0.742765,1.882585,5.531015,5.156267,6.109170,54.416236
1,2.323735,0.307302,0.441582,0.307302,0.650273,0.715858,0.313852,0.441582,2.341460,1.027927,...,11.798595,7.142534,1.885147,5.889316,1.808890,2.620070,21.975021,13.836777,25.897058,58.595875
2,1.179539,0.220547,0.459755,0.220547,0.561117,0.633178,0.224338,0.459755,0.866561,1.054159,...,7.452763,1.649575,1.438043,5.608336,0.987001,3.669196,11.129959,10.920092,13.316267,49.815057


In [185]:
raw_acc_data.filter(regex='girls',axis=1).columns[::15]

Index(['dry_education_allgirls_avg_adm3_hrs', 'childwalk_dry_education_allgirls_avg_adm3_hrs'], dtype='object')

In [186]:
df[unfiltered_acc_cols][:3]

,dry_District_HQs_avg_adm3,dry_education_allboys_avg_adm3,dry_education_allgirls_avg_adm3,dry_education_boys_avg_adm3,dry_education_boys_high_avg_adm3,dry_education_boys_middle_avg_adm3,dry_education_boys_primary_avg_adm3,dry_education_girls_avg_adm3,dry_education_girls_high_avg_adm3,dry_education_girls_middle_avg_adm3,...,childwalk_winter_health_family_avg_adm3,childwalk_winter_health_pharmacies_avg_adm3,childwalk_winter_health_primary_avg_adm3,childwalk_winter_health_private_avg_adm3,childwalk_winter_health_public_avg_adm3,childwalk_winter_health_secondary_avg_adm3,childwalk_winter_health_tertiary_avg_adm3,childwalk_winter_markets_All_avg_adm3,childwalk_winter_markets_Central_avg_adm3,childwalk_winter_Provincial_HQ_avg_adm3
0,0.962309,0.151154,0.182783,0.154368,0.324133,0.390303,0.159064,0.186505,0.458356,0.391861,...,5.626384,1.479962,0.847822,1.690228,0.742765,1.882585,5.531015,5.156267,6.109170,54.416236
1,2.323735,0.307302,0.441582,0.307302,0.650273,0.715858,0.313852,0.441582,2.341460,1.027927,...,11.798595,7.142534,1.885147,5.889316,1.808890,2.620070,21.975021,13.836777,25.897058,58.595875
2,1.179539,0.220547,0.459755,0.220547,0.561117,0.633178,0.224338,0.459755,0.866561,1.054159,...,7.452763,1.649575,1.438043,5.608336,0.987001,3.669196,11.129959,10.920092,13.316267,49.815057


In [188]:
# assign the three sets of access stats to separate lists

childwalk_cols = list(df.filter(regex='^(childwalk)',axis=1).columns)
walk_cols = list(df.filter(regex='^(walk)',axis=1).columns)
multimodal_cols = list(df.filter(regex='^(dry|msn|winter)',axis=1))
multimodal_cols = [item for item in multimodal_cols if not re.match('^(walk|childwalk)',item)]

# create a master list of all access cols

import itertools
acc_cols = list(itertools.chain(childwalk_cols,walk_cols,multimodal_cols))

# save out non access values to later join back in as needed

df_non_acc = df.drop(df[acc_cols].columns,axis=1)

In [189]:
childwalk_cols[::20]

['childwalk_dry_District_HQs_avg_adm3',
 'childwalk_dry_health_tertiary_avg_adm3',
 'childwalk_msn_health_primary_avg_adm3',
 'childwalk_winter_education_middle_avg_adm3']

In [190]:
walk_cols[::20]

['walk_dry_District_HQs_avg_adm3',
 'walk_dry_health_tertiary_avg_adm3',
 'walk_msn_health_primary_avg_adm3',
 'walk_winter_education_middle_avg_adm3']

In [191]:
multimodal_cols[::20]

['dry_District_HQs_avg_adm3',
 'dry_health_tertiary_avg_adm3',
 'msn_health_primary_avg_adm3',
 'winter_education_middle_avg_adm3']

In [192]:
# Children's walking speeds should be used for primary school access. This requires some adjustment

# create sets of the columns to add and remove
childwalk_ed_add = set([f'childwalk_dry_education_primary_avg_{adm_level}',f'childwalk_msn_education_primary_avg_{adm_level}',f'childwalk_winter_education_primary_avg_{adm_level}',
                        f'childwalk_dry_education_girls_primary_avg_{adm_level}',f'childwalk_msn_education_girls_primary_avg_{adm_level}',f'childwalk_winter_education_girls_primary_avg_{adm_level}',
                        f'childwalk_dry_education_boys_primary_avg_{adm_level}',f'childwalk_msn_education_boys_primary_avg_{adm_level}',f'childwalk_winter_education_boys_primary_avg_{adm_level}'])

ed_remove = set([f'dry_education_primary_avg_{adm_level}',f'childwalk_dry_education_middle_avg_{adm_level}',
f'msn_education_primary_avg_{adm_level}',f'childwalk_msn_education_middle_avg_{adm_level}',
f'winter_education_primary_avg_{adm_level}',f'childwalk_winter_education_middle_avg_{adm_level}',
f'dry_education_girls_primary_avg_{adm_level}',f'childwalk_dry_education_girls_middle_avg_{adm_level}',f'dry_education_girls_middle_avg_{adm_level}'
f'msn_education_girls_primary_avg_{adm_level}',f'childwalk_msn_education_girls_middle_avg_{adm_level}',f'msn_education_girls_middle_avg_{adm_level}'
f'winter_education_girls_primary_avg_{adm_level}',f'childwalk_winter_education_girls_middle_avg_{adm_level}' ,f'winter_education_girls_middle_avg_{adm_level}'
f'dry_education_boys_primary_avg_{adm_level}',f'childwalk_dry_education_boys_middle_avg_{adm_level}',f'dry_education_boys_middle_avg_{adm_level}'
f'msn_education_boys_primary_avg_{adm_level}',f'childwalk_msn_education_boys_middle_avg_{adm_level}',f'msn_education_boys_middle_avg_{adm_level}'
f'winter_education_boys_primary_avg_{adm_level}',f'childwalk_winter_education_boys_middle_avg_{adm_level}'])

# Remove and add these columns
standard_cols = list(set(multimodal_cols).difference(ed_remove))
standard_cols.extend(childwalk_ed_add)

In [193]:
standard_cols

['msn_education_allboys_avg_adm3',
 'dry_markets_Central_avg_adm3',
 'winter_health_pharmacies_avg_adm3',
 'winter_Provincial_HQ_avg_adm3',
 'msn_education_boys_avg_adm3',
 'dry_education_boys_avg_adm3',
 'dry_health_tertiary_avg_adm3',
 'msn_health_private_avg_adm3',
 'winter_education_girls_middle_avg_adm3',
 'msn_markets_All_avg_adm3',
 'winter_education_allgirls_avg_adm3',
 'dry_education_boys_primary_avg_adm3',
 'winter_health_public_avg_adm3',
 'winter_District_HQs_avg_adm3',
 'msn_health_public_avg_adm3',
 'dry_education_middle_avg_adm3',
 'winter_education_boys_avg_adm3',
 'winter_health_primary_avg_adm3',
 'dry_education_boys_middle_avg_adm3',
 'dry_health_family_avg_adm3',
 'msn_health_pharmacies_avg_adm3',
 'winter_education_girls_high_avg_adm3',
 'msn_education_boys_high_avg_adm3',
 'dry_markets_All_avg_adm3',
 'msn_education_girls_avg_adm3',
 'msn_Provincial_HQ_avg_adm3',
 'dry_education_allgirls_avg_adm3',
 'msn_education_girls_high_avg_adm3',
 'winter_health_tertiary_avg

In [194]:
# reduce the DF down to just identifying information, elevation, and the columns being used for analysis
df = pd.concat([df[(['Mean_elevation'] + list(adm_cols))],df[standard_cols]],axis=1,ignore_index=False)

In [195]:
df.head(3)

,Mean_elevation,ADM2_EN,Adm2_Code,ADM3_EN,Adm3_Code,msn_education_allboys_avg_adm3,dry_markets_Central_avg_adm3,winter_health_pharmacies_avg_adm3,winter_Provincial_HQ_avg_adm3,msn_education_boys_avg_adm3,...,msn_health_primary_avg_adm3,childwalk_msn_education_girls_primary_avg_adm3,childwalk_winter_education_girls_primary_avg_adm3,childwalk_winter_education_boys_primary_avg_adm3,childwalk_dry_education_primary_avg_adm3,childwalk_winter_education_primary_avg_adm3,childwalk_dry_education_girls_primary_avg_adm3,childwalk_msn_education_primary_avg_adm3,childwalk_msn_education_boys_primary_avg_adm3,childwalk_dry_education_boys_primary_avg_adm3
0,1317.718793,Abbottabad,PK201,Abbottabad,PK20101,0.197355,1.036626,0.548865,3.231324,0.201257,...,0.498859,0.427904,0.325190,0.259577,0.220939,0.221474,0.324649,0.289624,0.340336,0.258973
1,1757.084070,Batagram,PK203,Alai,PK20301,0.395936,3.402268,2.182935,6.103482,0.395936,...,1.243589,0.841776,0.685158,0.422137,0.347300,0.382421,0.642101,0.448827,0.499263,0.385195
2,1799.268152,Shangla,PK220,Alpuri,PK22001,0.287282,2.054575,0.934454,4.224185,0.287282,...,1.113937,0.932491,0.713813,0.272459,0.255985,0.258755,0.704829,0.334032,0.352236,0.269638


### Prepare seasonal master values

### Education overall

Education is a bit complicated because of Transport's requirements. First we have to define our custom weighting schemas and the main columns to operate on

In [196]:
# equal weighting schema for comparison's sake

educ_equal_wts = np.array([1,1,1,1,1,1]) / 6

# set up weighting schema to prioritize lower schools and girls' education

girls_wts = np.array([.4,.4,.2]) * (2/3)
boys_wts = np.array([.4,.4,.2]) * (1/3)

# put the girls/boys arrays together
educ_mast_wts = np.concatenate((girls_wts,boys_wts),axis=0)
educ_mast_wts

array([0.26666667, 0.26666667, 0.13333333, 0.13333333, 0.13333333,
       0.06666667])

In [197]:
# because we mix childwalking and multimodal for education's master value it's easiest just to manually specify which columns to use
educ_mast_cols = [f'childwalk_dry_education_girls_primary_avg_{adm_level}',\
                  f'dry_education_girls_middle_avg_{adm_level}',\
                  f'dry_education_girls_high_avg_{adm_level}',\
                  f'childwalk_dry_education_boys_primary_avg_{adm_level}',\
                  f'dry_education_boys_middle_avg_{adm_level}',\
                  f'dry_education_boys_high_avg_{adm_level}']

Convert every column to an index (so features with naturally higher travel times, like hospitals, don't unduly  weight results)

In [198]:
# OLD

filtered_acc_cols = list(df.filter(regex='^(dry|msn|winter)',axis=1).columns) # only the access columns remaining after we remove those note used for index calcs

max_acc = np.array(np.max(df[filtered_acc_cols],axis=0)) # column-wise max

df[filtered_acc_cols] = df[filtered_acc_cols] / max_acc # divide by max for column-wise index
df[filtered_acc_cols][:3]

,msn_education_allboys_avg_adm3,dry_markets_Central_avg_adm3,winter_health_pharmacies_avg_adm3,winter_Provincial_HQ_avg_adm3,msn_education_boys_avg_adm3,dry_education_boys_avg_adm3,dry_health_tertiary_avg_adm3,msn_health_private_avg_adm3,winter_education_girls_middle_avg_adm3,msn_markets_All_avg_adm3,...,msn_education_girls_primary_avg_adm3,winter_education_high_avg_adm3,winter_health_secondary_avg_adm3,dry_education_boys_high_avg_adm3,dry_health_primary_avg_adm3,dry_education_allboys_avg_adm3,winter_health_private_avg_adm3,dry_education_girls_avg_adm3,winter_education_boys_primary_avg_adm3,msn_health_primary_avg_adm3
0,0.081887,0.147385,0.096237,0.320855,0.083506,0.085599,0.146053,0.096455,0.118018,0.139369,...,0.105819,0.095817,0.155449,0.115635,0.084905,0.083817,0.107734,0.104402,0.091054,0.083194
1,0.164282,0.483728,0.382751,0.606046,0.164282,0.170402,0.449983,0.300799,0.336310,0.443742,...,0.240531,0.211866,0.290766,0.231985,0.213255,0.170402,0.366206,0.247190,0.195349,0.207390
2,0.119200,0.292115,0.163845,0.419441,0.119200,0.122296,0.282764,0.265521,0.334795,0.272303,...,0.259872,0.174055,0.316217,0.200179,0.191433,0.122296,0.321162,0.257362,0.126312,0.185769


In [199]:
# # NEW -- have lower values represent worse index, and vice-versa
# # filter columns
# filtered_acc_cols = list(df.filter(regex='^(dry|msn|winter)',axis=1).columns) # only the access columns remaining after we remove those note used for index calcs

# # compute min and max for indexing calculations
# min_acc = np.array(np.min(df[filtered_acc_cols],axis=0)) # column-wise min
# max_acc = np.array(np.max(df[filtered_acc_cols],axis=0)) # column-wise max

# # calculate highest TT value as worst (lowest) index value and lowest TT as best (1)
# df[filtered_acc_cols] = (max_acc - df[filtered_acc_cols]) / (max_acc - min_acc)
# df[filtered_acc_cols][:3]

In [200]:
np.max(df[filtered_acc_cols].iloc[:,4])

1.0

In [201]:
df = pd.concat([df,raw_acc_data],axis=1,ignore_index=False)

Now consolidate into master values, weighting appropriately

In [202]:
# education by levels

df['dry_educ_idx'] = np.nansum((df[educ_mast_cols] * educ_mast_wts),axis=1)

df['msn_educ_idx'] = np.nansum((df[[col.replace('dry','msn') for col in educ_mast_cols]] * educ_mast_wts),axis=1) # use the same mast_cols list, but with monsoon data instead

df['winter_educ_idx'] = np.nansum((df[[col.replace('dry','winter') for col in educ_mast_cols]] * educ_mast_wts),axis=1)

# education weighting equally -- for comparison's sake

df['dry_educ_eqwt_idx'] = np.nansum((df[educ_mast_cols] * educ_equal_wts),axis=1)

df['msn_educ_eqwt_idx'] = np.nansum((df[[col.replace('dry','msn') for col in educ_mast_cols]] * educ_equal_wts),axis=1)

df['winter_educ_eqwt_idx'] = np.nansum((df[[col.replace('dry','winter') for col in educ_mast_cols]] * educ_equal_wts),axis=1)


**Optional**</br>
Sensitivity test educational weighting schemes

In [203]:
wt_samp_arr, vals_arr, ranks_arr = st.Sensitivity_weighting(df,educ_mast_cols,iterations=500000)

Calculate descriptive stats for sensitivity tested rankings of education per admin

In [204]:
# compute basic stats for each entity's ranking
# axis=0 for operating by column
ranks_mode = stats.mode(ranks_arr,axis=0)
ranks_mean = np.mean(ranks_arr,axis=0)
ranks_std = np.std(ranks_arr,axis=0)
vals_mean = np.sum(np.mean(vals_arr,axis=0),axis=1)

In [205]:
ranks_std

array([ 1.46197772,  4.62022571,  3.48475789,  5.09786243,  3.01145627,
        2.96247767,  2.17360117,  2.84003403,  5.19039719, 10.76744764,
        2.04005043,  4.81038535,  3.95086493,  6.85094787,  2.37695023,
        3.07886541,  1.8631911 ,  6.24776237,  0.68803087,  6.22645424,
        2.29113725,  3.4224893 ,  4.52865363,  0.66889761,  2.13367863,
        3.47865207,  5.55820992,  4.81168225,  6.44508872,  0.49046452,
        1.64426333,  3.79181509,  3.52846252,  1.65544437,  2.59457089,
        1.41649029,  4.84615891,  1.38126216,  2.53421952,  4.66751363,
        2.68578816,  2.60324804,  2.28649229,  6.56627204,  2.70887425,
        4.55729029,  5.05333887,  2.4271285 ,  3.52928524,  5.30590014,
        4.65222684,  3.42985246,  1.54374559,  2.83111231,  3.96903231,
        3.85851206,  2.75764782,  1.30444557,  2.05770682,  6.49494419,
        3.05572744,  4.87784174,  2.22628429,  4.00826746,  4.10259313,
        1.97666418,  2.40616477,  3.97385037,  5.85081913,  1.81

In [206]:
# join in the key stats for sensitivity tested educational rankings
df['sens_test_educ_rank_mean'] = ranks_mean
df['sens_test_educ_rank_mode'] = ranks_mode[0][0]
df['sens_test_educ_rank_std'] = ranks_std

In [207]:
# Now calculate the same for an equal weighting schema and the unbalanced weighting scheme requested by Transport

df['educ_eqwt_rank'] = st.rank_by_weight(df,educ_mast_cols,educ_equal_wts)
df['educ_eqwt_val'] = np.nansum(df[educ_mast_cols] * educ_equal_wts, axis=1)

df['educ_altwt_rank'] = st.rank_by_weight(df,educ_mast_cols,educ_mast_wts)
df['educ_altwt_val'] = np.nansum(df[educ_mast_cols] * educ_mast_wts, axis=1)

df['educ_sens_test_val_mean'] = vals_mean

In [208]:
df[['sens_test_educ_rank_mean','sens_test_educ_rank_mode','sens_test_educ_rank_std']].head(10)

,sens_test_educ_rank_mean,sens_test_educ_rank_mode,sens_test_educ_rank_std
0,83.098342,83,1.461978
1,30.541114,34,4.620226
2,42.255332,44,3.484758
3,10.952118,8,5.097862
4,102.184052,101,3.011456
5,26.286716,27,2.962478
6,45.377338,45,2.173601
7,61.259986,62,2.840034
8,109.962330,115,5.190397
9,36.844632,35,10.767448


In [209]:
df[['educ_sens_test_val_mean','educ_eqwt_val','educ_altwt_val']].head(10)

,educ_sens_test_val_mean,educ_eqwt_val,educ_altwt_val
0,0.179418,0.179369,0.193483
1,0.407446,0.407313,0.433797
2,0.322584,0.322502,0.377397
3,0.604815,0.604584,0.699454
4,0.144362,0.144303,0.159709
5,0.439198,0.439081,0.475255
6,0.300836,0.300752,0.333664
7,0.240328,0.240288,0.271880
8,0.129015,0.128987,0.135998
9,0.370882,0.370656,0.463348


In [210]:
# join in the key stats for sensitivity tested educational rankings
df['sens_test_educ_rank_mean'] = ranks_mean
df['sens_test_educ_rank_mode'] = ranks_mode[0][0]
df['sens_test_educ_rank_std'] = ranks_std

In [211]:
# Now calculate the same for an equal weighting schema and the unbalanced weighting scheme requested by Transport

df['educ_eqwt_rank'] = st.rank_by_weight(df,educ_mast_cols,educ_equal_wts)
df['educ_eqwt_val'] = np.nansum(df[educ_mast_cols] * educ_equal_wts, axis=1)

df['educ_altwt_rank'] = st.rank_by_weight(df,educ_mast_cols,educ_mast_wts)
df['educ_altwt_val'] = np.nansum(df[educ_mast_cols] * educ_mast_wts, axis=1)

df['educ_sens_test_val_mean'] = vals_mean

In [212]:
df[['sens_test_educ_rank_mean','sens_test_educ_rank_mode','sens_test_educ_rank_std']].head(10)

,sens_test_educ_rank_mean,sens_test_educ_rank_mode,sens_test_educ_rank_std
0,83.098342,83,1.461978
1,30.541114,34,4.620226
2,42.255332,44,3.484758
3,10.952118,8,5.097862
4,102.184052,101,3.011456
5,26.286716,27,2.962478
6,45.377338,45,2.173601
7,61.259986,62,2.840034
8,109.962330,115,5.190397
9,36.844632,35,10.767448


In [213]:
df[['educ_sens_test_val_mean','educ_eqwt_val','educ_altwt_val']].head(10)

,educ_sens_test_val_mean,educ_eqwt_val,educ_altwt_val
0,0.179418,0.179369,0.193483
1,0.407446,0.407313,0.433797
2,0.322584,0.322502,0.377397
3,0.604815,0.604584,0.699454
4,0.144362,0.144303,0.159709
5,0.439198,0.439081,0.475255
6,0.300836,0.300752,0.333664
7,0.240328,0.240288,0.271880
8,0.129015,0.128987,0.135998
9,0.370882,0.370656,0.463348


#### Other destinations

Health, markets, and administrative access are more straightforward to consolidate

In [214]:
# health by levels

df[f'dry_health_idx'] = np.mean(df[[f'dry_health_primary_avg_{adm_level}',f'dry_health_secondary_avg_{adm_level}',f'dry_health_tertiary_avg_{adm_level}']],axis=1) # using np.mean implies equal weighting of sub-categories
df[f'msn_health_idx'] = np.mean(df[[f'msn_health_primary_avg_{adm_level}',f'msn_health_secondary_avg_{adm_level}',f'msn_health_tertiary_avg_{adm_level}']],axis=1)
df[f'winter_health_idx'] = np.mean(df[[f'winter_health_primary_avg_{adm_level}',f'winter_health_secondary_avg_{adm_level}',f'winter_health_tertiary_avg_{adm_level}']],axis=1)

In [215]:
# markets
df[f'dry_markets_idx'] = np.mean(df[[f'dry_markets_All_avg_{adm_level}',f'dry_markets_Central_avg_{adm_level}']],axis=1) # using np.mean implies equal weighting of sub-categories
df[f'msn_markets_idx'] = np.mean(df[[f'msn_markets_All_avg_{adm_level}',f'msn_markets_Central_avg_{adm_level}']],axis=1)
df[f'winter_markets_idx'] = np.mean(df[[f'winter_markets_All_avg_{adm_level}',f'winter_markets_Central_avg_{adm_level}']],axis=1)


In [216]:
# administrative
# I assume that access to the Provincial HQ is not materially significant for administrative functions -- therefore this is just District HQ access

df[f'dry_admin_idx'] = df[f'dry_District_HQs_avg_{adm_level}']
df[f'msn_admin_idx'] = df[f'msn_District_HQs_avg_{adm_level}']
df[f'winter_admin_idx'] = df[f'winter_District_HQs_avg_{adm_level}']


#### Prepare categorical and overall master values, weighted by season

In [217]:
# equal weights for variables
eq_wts = [0.3333, 0.3333, 0.3333]

In [218]:

# categorize admins  by the population-weighted mean elevation of populated places in that tehsil
df['Elevation_category'] = pd.cut(df['Mean_elevation'],bins=[0,1500,2250,100000],labels=['Low','Medium','High'])

# Define the weighting of a season according to the tehsils's classification

season_wts_dct = {
    float("NaN") : np.array([0.3333,0.3333,0.3333]),
    'Low' : np.array([0.3333,0.3333,0.3333]),
    'Medium' : np.array([0.2667,0.3333,0.4]),
    'High' : np.array([0.25,0.25,0.5])   
}

# Assign the seasonal weighting as a list, for later use
df['seasonal_wts'] = df['Elevation_category'].map(season_wts_dct)
seasonal_wts_arr = np.stack(df['seasonal_wts']) # turn the Series of weights into an array of shape (3,number_of_admins) so it can be multiplied by the 3 seasonal values for each category

TypeError: unhashable type: 'numpy.ndarray'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


Master indices

In [219]:
# calculate the master value for each feature type by weighting each tehsil's seasonal master values by its seasonal weights
df['educ_altwt_idx'] = np.nansum(np.multiply(df[['dry_educ_idx','msn_educ_idx','winter_educ_idx']],seasonal_wts_arr),axis=1)
df['educ_eqwt_idx'] = np.nansum(np.multiply(df[['dry_educ_eqwt_idx','msn_educ_eqwt_idx','winter_educ_eqwt_idx']],seasonal_wts_arr),axis=1)
df['health_idx'] = np.nansum(np.multiply(df[['dry_health_idx','msn_health_idx','winter_health_idx']],seasonal_wts_arr),axis=1)
df['markets_idx'] = np.nansum(np.multiply(df[['dry_markets_idx','msn_markets_idx','winter_markets_idx']],seasonal_wts_arr),axis=1)
df['admin_idx'] = np.nansum(np.multiply(df[['dry_admin_idx','msn_admin_idx','winter_admin_idx']],seasonal_wts_arr),axis=1)


#### Specialty educational index calculations

In [220]:
# Differences in index values

# overall index vals -- with and without private schools

df[f'dry_educ_alllevels_gender_dif'] = df[f'dry_education_girls_avg_{adm_level}'] - df[f'dry_education_boys_avg_{adm_level}']
df[f'msn_educ_alllevels_gender_dif'] = df[f'msn_education_girls_avg_{adm_level}'] - df[f'msn_education_boys_avg_{adm_level}']
df[f'winter_educ_alllevels_gender_dif'] = df[f'winter_education_girls_avg_{adm_level}'] - df[f'winter_education_boys_avg_{adm_level}']

df[f'dry_educ_alllevels_w_privschool_dif'] = df[f'dry_education_allgirls_avg_{adm_level}'] - df[f'dry_education_allboys_avg_{adm_level}']
df[f'msn_educ_alllevels_w_privschool_dif'] = df[f'msn_education_allgirls_avg_{adm_level}'] - df[f'msn_education_allboys_avg_{adm_level}']
df[f'winter_educ_alllevels_w_privschool_dif'] = df[f'winter_education_allgirls_avg_{adm_level}'] - df[f'winter_education_allboys_avg_{adm_level}']

# primary

df[f'childwalk_dry_educ_primary_gender_dif'] = df[f'childwalk_dry_education_girls_primary_avg_{adm_level}'] - df[f'childwalk_dry_education_boys_primary_avg_{adm_level}']
df[f'childwalk_msn_educ_primary_gender_dif'] = df[f'childwalk_msn_education_girls_primary_avg_{adm_level}'] - df[f'childwalk_msn_education_boys_primary_avg_{adm_level}']
df[f'childwalk_winter_educ_primary_gender_dif'] = df[f'childwalk_winter_education_girls_primary_avg_{adm_level}'] - df[f'childwalk_winter_education_boys_primary_avg_{adm_level}']

# middle

df[f'dry_educ_middle_gender_dif'] = df[f'dry_education_girls_middle_avg_{adm_level}'] - df[f'dry_education_boys_middle_avg_{adm_level}']
df[f'msn_educ_middle_gender_dif'] = df[f'msn_education_girls_middle_avg_{adm_level}'] - df[f'msn_education_boys_middle_avg_{adm_level}']
df[f'winter_educ_middle_gender_dif'] = df[f'winter_education_girls_middle_avg_{adm_level}'] - df[f'winter_education_boys_middle_avg_{adm_level}']

# high

df[f'dry_educ_high_gender_dif'] = df[f'dry_education_girls_high_avg_{adm_level}'] - df[f'dry_education_boys_high_avg_{adm_level}']
df[f'msn_educ_high_gender_dif'] = df[f'msn_education_girls_high_avg_{adm_level}'] - df[f'msn_education_boys_high_avg_{adm_level}']
df[f'winter_educ_high_gender_dif'] = df[f'winter_education_girls_high_avg_{adm_level}'] - df[f'winter_education_boys_high_avg_{adm_level}']


In [221]:
# Differences in hours

# overall in hours -- with and without private schools

df[f'dry_educ_gender_dif_hrs'] = df[f'dry_education_girls_avg_{adm_level}_hrs'] - df[f'dry_education_boys_avg_{adm_level}_hrs']
df[f'msn_educ_gender_dif_hrs'] = df[f'msn_education_girls_avg_{adm_level}_hrs'] - df[f'msn_education_boys_avg_{adm_level}_hrs']
df[f'winter_educ_gender_dif_hrs'] = df[f'winter_education_girls_avg_{adm_level}_hrs'] - df[f'winter_education_boys_avg_{adm_level}_hrs']

df[f'dry_educ_allgender_dif_hrs'] = df[f'dry_education_allgirls_avg_{adm_level}_hrs'] - df[f'dry_education_allboys_avg_{adm_level}_hrs']
df[f'msn_educ_allgender_dif_hrs'] = df[f'msn_education_allgirls_avg_{adm_level}_hrs'] - df[f'msn_education_allboys_avg_{adm_level}_hrs']
df[f'winter_educ_allgender_dif_hrs'] = df[f'winter_education_allgirls_avg_{adm_level}_hrs'] - df[f'winter_education_allboys_avg_{adm_level}_hrs']

# primary

df[f'childwalk_dry_educ_primary_gender_dif_hrs'] = df[f'childwalk_dry_education_girls_primary_avg_{adm_level}_hrs'] - df[f'childwalk_dry_education_boys_primary_avg_{adm_level}_hrs']
df[f'childwalk_msn_educ_primary_gender_dif_hrs'] = df[f'childwalk_msn_education_girls_primary_avg_{adm_level}_hrs'] - df[f'childwalk_msn_education_boys_primary_avg_{adm_level}_hrs']
df[f'childwalk_winter_educ_primary_gender_dif_hrs'] = df[f'childwalk_winter_education_girls_primary_avg_{adm_level}_hrs'] - df[f'childwalk_winter_education_boys_primary_avg_{adm_level}_hrs']

# middle

df[f'dry_educ_middle_gender_dif_hrs'] = df[f'dry_education_girls_middle_avg_{adm_level}_hrs'] - df[f'dry_education_boys_middle_avg_{adm_level}_hrs']
df[f'msn_educ_middle_gender_dif_hrs'] = df[f'msn_education_girls_middle_avg_{adm_level}_hrs'] - df[f'msn_education_boys_middle_avg_{adm_level}_hrs']
df[f'winter_educ_middle_gender_dif_hrs'] = df[f'winter_education_girls_middle_avg_{adm_level}_hrs'] - df[f'winter_education_boys_middle_avg_{adm_level}_hrs']

# high

df[f'dry_educ_high_gender_dif_hrs'] = df[f'dry_education_girls_high_avg_{adm_level}_hrs'] - df[f'dry_education_boys_high_avg_{adm_level}_hrs']
df[f'msn_educ_high_gender_dif_hrs'] = df[f'msn_education_girls_high_avg_{adm_level}_hrs'] - df[f'msn_education_boys_high_avg_{adm_level}_hrs']
df[f'winter_educ_high_gender_dif_hrs'] = df[f'winter_education_girls_high_avg_{adm_level}_hrs'] - df[f'winter_education_boys_high_avg_{adm_level}_hrs']


In [222]:
# use dry only as we'll replace inline below

educ_dif_cols = ['childwalk_dry_educ_primary_gender_dif',
 'dry_educ_middle_gender_dif',
 'dry_educ_high_gender_dif']

# create weights for just three columns

educ_dif_wts = [0.4,0.4,0.2]
educ_eq_dif_wts = [1/3, 1/3, 1/3]

In [223]:
# education access differences by levels

df['dry_educ_gender_dif_idx'] = np.nansum((df[educ_dif_cols] * educ_dif_wts),axis=1)
df['msn_educ_gender_dif_idx'] = np.nansum((df[[col.replace('dry','msn') for col in educ_dif_cols]] * educ_dif_wts),axis=1) # use the same dif_cols list, but with monsoon data instead
df['winter_educ_gender_dif_idx'] = np.nansum((df[[col.replace('dry','winter') for col in educ_dif_cols]] * educ_dif_wts),axis=1)

# education access differences weighting equally -- for comparison's sake

df['dry_educ_eqwt_gender_dif_idx'] = np.nansum((df[educ_dif_cols] * educ_eq_dif_wts),axis=1)
df['msn_educ_eqwt_gender_dif_idx'] = np.nansum((df[[col.replace('dry','msn') for col in educ_dif_cols]] * educ_eq_dif_wts),axis=1)
df['winter_educ_eqwt_gender_dif_idx'] = np.nansum((df[[col.replace('dry','winter') for col in educ_dif_cols]] * educ_eq_dif_wts),axis=1)


In [224]:
# Differences in hours

# overall in hours -- with and without private schools

df[f'dry_educ_gender_dif_hrs'] = df[f'dry_education_girls_avg_{adm_level}_hrs'] - df[f'dry_education_boys_avg_{adm_level}_hrs']
df[f'msn_educ_gender_dif_hrs'] = df[f'msn_education_girls_avg_{adm_level}_hrs'] - df[f'msn_education_boys_avg_{adm_level}_hrs']
df[f'winter_educ_gender_dif_hrs'] = df[f'winter_education_girls_avg_{adm_level}_hrs'] - df[f'winter_education_boys_avg_{adm_level}_hrs']

df[f'dry_educ_allgender_dif_hrs'] = df[f'dry_education_allgirls_avg_{adm_level}_hrs'] - df[f'dry_education_allboys_avg_{adm_level}_hrs']
df[f'msn_educ_allgender_dif_hrs'] = df[f'msn_education_allgirls_avg_{adm_level}_hrs'] - df[f'msn_education_allboys_avg_{adm_level}_hrs']
df[f'winter_educ_allgender_dif_hrs'] = df[f'winter_education_allgirls_avg_{adm_level}_hrs'] - df[f'winter_education_allboys_avg_{adm_level}_hrs']

# primary

df[f'childwalk_dry_educ_primary_gender_dif_hrs'] = df[f'childwalk_dry_education_girls_primary_avg_{adm_level}_hrs'] - df[f'childwalk_dry_education_boys_primary_avg_{adm_level}_hrs']
df[f'childwalk_msn_educ_primary_gender_dif_hrs'] = df[f'childwalk_msn_education_girls_primary_avg_{adm_level}_hrs'] - df[f'childwalk_msn_education_boys_primary_avg_{adm_level}_hrs']
df[f'childwalk_winter_educ_primary_gender_dif_hrs'] = df[f'childwalk_winter_education_girls_primary_avg_{adm_level}_hrs'] - df[f'childwalk_winter_education_boys_primary_avg_{adm_level}_hrs']

# middle

df[f'dry_educ_middle_gender_dif_hrs'] = df[f'dry_education_girls_middle_avg_{adm_level}_hrs'] - df[f'dry_education_boys_middle_avg_{adm_level}_hrs']
df[f'msn_educ_middle_gender_dif_hrs'] = df[f'msn_education_girls_middle_avg_{adm_level}_hrs'] - df[f'msn_education_boys_middle_avg_{adm_level}_hrs']
df[f'winter_educ_middle_gender_dif_hrs'] = df[f'winter_education_girls_middle_avg_{adm_level}_hrs'] - df[f'winter_education_boys_middle_avg_{adm_level}_hrs']

# high

df[f'dry_educ_high_gender_dif_hrs'] = df[f'dry_education_girls_high_avg_{adm_level}_hrs'] - df[f'dry_education_boys_high_avg_{adm_level}_hrs']
df[f'msn_educ_high_gender_dif_hrs'] = df[f'msn_education_girls_high_avg_{adm_level}_hrs'] - df[f'msn_education_boys_high_avg_{adm_level}_hrs']
df[f'winter_educ_high_gender_dif_hrs'] = df[f'winter_education_girls_high_avg_{adm_level}_hrs'] - df[f'winter_education_boys_high_avg_{adm_level}_hrs']


In [225]:
# Education w/ and w/out private schools, per gender

## BY INDEX VALUES
df['educ_girls_alllevels_pubschool_idx'] = np.nansum(df[[f'dry_education_girls_avg_{adm_level}',f'msn_education_girls_avg_{adm_level}',f'winter_education_girls_avg_{adm_level}']] * seasonal_wts_arr,axis=1)

df['educ_boys_alllevels_pubschool_idx'] = np.nansum(df[[f'dry_education_boys_avg_{adm_level}',f'msn_education_boys_avg_{adm_level}',f'winter_education_boys_avg_{adm_level}']] * seasonal_wts_arr,axis=1)

df['educ_girls_alllevels_w_privschool_idx'] = np.nansum(df[[f'dry_education_allgirls_avg_{adm_level}',f'msn_education_allgirls_avg_{adm_level}',f'winter_education_allgirls_avg_{adm_level}']] * seasonal_wts_arr,axis=1)

df['educ_boys_alllevels_w_privschool_idx'] = np.nansum(df[[f'dry_education_allboys_avg_{adm_level}',f'msn_education_allboys_avg_{adm_level}',f'winter_education_allboys_avg_{adm_level}']] * seasonal_wts_arr,axis=1)

## BY HOURS

df['educ_girls_alllevels_pubschool_hrs'] = np.nansum(df[[f'dry_education_girls_avg_{adm_level}_hrs',f'msn_education_girls_avg_{adm_level}_hrs',f'winter_education_girls_avg_{adm_level}_hrs']] * seasonal_wts_arr,axis=1)

df['educ_boys_alllevels_pubschool_hrs'] = np.nansum(df[[f'dry_education_boys_avg_{adm_level}_hrs',f'msn_education_boys_avg_{adm_level}_hrs',f'winter_education_boys_avg_{adm_level}_hrs']] * seasonal_wts_arr,axis=1)

df['educ_girls_alllevels_w_privschool_hrs'] = np.nansum(df[[f'dry_education_allgirls_avg_{adm_level}_hrs',f'msn_education_allgirls_avg_{adm_level}_hrs',f'winter_education_allgirls_avg_{adm_level}_hrs']] * seasonal_wts_arr,axis=1)

df['educ_boys_alllevels_w_privschool_hrs'] = np.nansum(df[[f'dry_education_allboys_avg_{adm_level}_hrs',f'msn_education_allboys_avg_{adm_level}_hrs',f'winter_education_allboys_avg_{adm_level}_hrs']] * seasonal_wts_arr,axis=1)

In [226]:
# Calculate educational difference indices, weighting by seasons.

## One index
df['educ_gender_dif_idx'] = np.nansum(np.multiply(df[['dry_educ_gender_dif_idx',\
                                                      'msn_educ_gender_dif_idx',\
                                                      'winter_educ_gender_dif_idx']],\
                                                  seasonal_wts_arr),axis=1)

df['educ_gender_eqwt_dif_idx'] = np.nansum(np.multiply(df[['dry_educ_eqwt_gender_dif_idx',\
                                                           'msn_educ_eqwt_gender_dif_idx',\
                                                           'winter_educ_eqwt_gender_dif_idx']],\
                                                  seasonal_wts_arr),axis=1)

## Differences in hours
df['educ_primary_gender_dif_hrs'] = np.nansum(np.multiply(df[['childwalk_dry_educ_primary_gender_dif_hrs',\
                                                              'childwalk_msn_educ_primary_gender_dif_hrs',\
                                                              'childwalk_winter_educ_primary_gender_dif_hrs']],seasonal_wts_arr),axis=1)

df['educ_middle_gender_dif_hrs'] = np.nansum(np.multiply(df[['dry_educ_middle_gender_dif_hrs',\
                                                              'msn_educ_middle_gender_dif_hrs',\
                                                              'winter_educ_middle_gender_dif_hrs']],seasonal_wts_arr),axis=1)


df['educ_high_gender_dif_hrs'] = np.nansum(np.multiply(df[['dry_educ_high_gender_dif_hrs',\
                                                              'msn_educ_high_gender_dif_hrs',\
                                                              'winter_educ_high_gender_dif_hrs']],seasonal_wts_arr),axis=1)

# Overall difference indices for all levels,w/out and w/ private schools included

df['educ_alllevels_dif'] = np.nansum(np.multiply(df[[f'dry_educ_alllevels_gender_dif',\
                                                              f'msn_educ_alllevels_gender_dif',\
                                                              f'winter_educ_alllevels_gender_dif']],seasonal_wts_arr),axis=1)

df['educ_alllevels_w_privschool_dif'] = np.nansum(np.multiply(df[[f'dry_educ_alllevels_w_privschool_dif',\
                                                              f'msn_educ_alllevels_w_privschool_dif',\
                                                              f'winter_educ_alllevels_w_privschool_dif']],seasonal_wts_arr),axis=1)

#### Finalize

Check out the indices

In [227]:
df[['educ_eqwt_idx','health_idx','markets_idx','admin_idx']].tail(5)

,educ_eqwt_idx,health_idx,markets_idx,admin_idx
111,0.281956,0.221387,0.234534,0.317991
112,0.438344,0.349488,0.358100,0.201213
113,0.200246,0.169680,0.231651,0.300865
114,0.476753,0.355648,0.363947,0.476075
115,0.187008,0.114979,0.100542,0.162541


In [228]:
# normalize all values to enable comparability

df['educ_altwt_idx'] = df['educ_altwt_idx'] / np.max(df['educ_altwt_idx'])
df['educ_eqwt_idx'] = df['educ_eqwt_idx'] / np.max(df['educ_eqwt_idx'])
df['health_idx'] = df['health_idx'] / np.max(df['health_idx'])
df['markets_idx'] = df['markets_idx'] / np.max(df['markets_idx'])
df['admin_idx'] = df['admin_idx'] / np.max(df['admin_idx'])


Finally, rank the admins by their overall inaccessibility index, weighting each of education, health, and markets equally (admin_idx is held out for separate analysis)

In [229]:
# weight education double relative to other sectors
educ_priority_wts = [0.5,0.25,0.25]

# identify rank according to equal weighting schemes

df['overall_eq_wt_rank'] = st.rank_by_weight(df,['educ_eqwt_idx','health_idx','markets_idx'],eq_wts)
df['overall_eq_wt_idx'] = np.nansum(df[['educ_eqwt_idx','health_idx','markets_idx']] * eq_wts, axis=1)

df['overall_educ_altwt_rank'] = st.rank_by_weight(df,['educ_altwt_idx','health_idx','markets_idx'],eq_wts)
df['overall_educ_altwt_idx'] = np.nansum(df[['educ_altwt_idx','health_idx','markets_idx']] * eq_wts, axis=1)

df['overall_educ_priority_wt_rank'] = st.rank_by_weight(df,['educ_altwt_idx','health_idx','markets_idx'],educ_priority_wts)
df['overall_educ_priority_wt_idx'] = np.nansum(df[['educ_altwt_idx','health_idx','markets_idx']] * educ_priority_wts, axis=1)


In [230]:
df[['educ_eqwt_idx','educ_altwt_idx','health_idx','markets_idx']].tail(5)

,educ_eqwt_idx,educ_altwt_idx,health_idx,markets_idx
111,0.209200,0.203960,0.247358,0.238550
112,0.325233,0.339424,0.390488,0.364231
113,0.148574,0.151955,0.189586,0.235617
114,0.353731,0.347462,0.397371,0.370178
115,0.138752,0.136410,0.128468,0.102264


#### Find top N% for overall and each index

In [231]:
def col_pctile(df,col):
    
    col_pctile = df[col].rank(method='max').apply(lambda x: 100.0 * (x-1) / (df[col].size -1))
    
    return col_pctile

In [232]:
df['educ_ewqt_pctile'] = col_pctile(df,'educ_eqwt_idx')
df['educ_altwt_pctile'] = col_pctile(df,'educ_altwt_idx')
df['health_pctile'] = col_pctile(df,'health_idx')
df['markets_pctile'] = col_pctile(df,'markets_idx')
df['admin_pctile'] = col_pctile(df,'admin_idx')
df['overall_pctile'] = col_pctile(df,'overall_eq_wt_idx')
df['overall_educpriority_pctile'] = col_pctile(df,'overall_educ_priority_wt_idx')

In [233]:
pctile_cols = ['educ_altwt_pctile','health_pctile','markets_pctile','admin_pctile']

In [234]:
df[pctile_cols]

,educ_altwt_pctile,health_pctile,markets_pctile,admin_pctile
0,29.565217,29.565217,26.956522,42.608696
1,73.043478,67.826087,86.956522,86.086957
2,67.826087,64.347826,63.478261,57.391304
3,94.782609,80.000000,70.434783,76.521739
4,13.043478,0.869565,0.000000,0.000000
...,...,...,...,...
111,49.565217,59.130435,53.043478,59.130435
112,75.652174,73.043478,74.782609,33.043478
113,32.173913,45.217391,51.304348,53.913043
114,76.521739,75.652174,75.652174,77.391304


In [235]:
pctiles_20pct = np.digitize(df[pctile_cols], bins = [0,80,100])
pctiles_20pct_overall = np.nansum(np.where(pctiles_20pct == 2, 1, 0),axis=1) / pctiles_20pct.shape[1]

In [236]:
df['deprivation_20pct'] = pctiles_20pct_overall

#### Append focus district yes/no info

In [237]:
non_focus_adm2_lst = ['PK201','PK204','PK205','PK241','PK243','PK209','PK211','PK215','PK216','PK217','PK218','PK219','PK221','PK222']

df['Adm2_Focus'] = 'Yes'
df.loc[df['Adm2_Code'].isin(non_focus_adm2_lst),'Adm2_Focus'] = 'No'

In [238]:
df[['Adm2_Code','Adm2_Focus']]

,Adm2_Code,Adm2_Focus
0,PK201,No
1,PK203,Yes
2,PK220,Yes
3,PK235,Yes
4,PK222,No
...,...,...
111,PK232,Yes
112,PK238,Yes
113,PK230,Yes
114,PK239,Yes


In [239]:
# checking this worked
df['Adm2_Focus'].unique()

array(['No', 'Yes'], dtype=object)

TEMP -- Quick analysis of all schools vs. just public schools results

In [240]:
pubschool_girls = df['educ_girls_alllevels_pubschool_idx'] - df['educ_girls_alllevels_w_privschool_idx']
pubschool_boys = df['educ_boys_alllevels_pubschool_idx'] - df['educ_boys_alllevels_w_privschool_idx']

allschools_girls = (df['educ_girls_alllevels_pubschool_hrs'] - df['educ_girls_alllevels_w_privschool_hrs']) * 60
allschools_boys = (df['educ_boys_alllevels_pubschool_hrs'] - df['educ_boys_alllevels_w_privschool_hrs']) * 60

### Export

Export tabular data for charting, sensitivity analysis, etc.

In [242]:
df.sort_values('overall_eq_wt_rank').to_csv(os.path.join(data_dir,tab_dir,f"final//{adm_level}_idx_access_stats_{today}.csv"),index=False)

Export shapefile for use elsewhere

In [243]:
# export tehsils to geographic file
if adm_level == 'adm3':
    
    # arrange
    adm3_geo = pd.merge(df[(['ADM2_EN','Adm2_Code',
           'Elevation_category',
           'dry_educ_idx', 'msn_educ_idx','winter_educ_idx',
           'educ_gender_dif_idx','educ_gender_eqwt_dif_idx',
           'educ_primary_gender_dif_hrs','educ_middle_gender_dif_hrs','educ_high_gender_dif_hrs',
           'dry_educ_alllevels_gender_dif','msn_educ_alllevels_gender_dif', 'winter_educ_alllevels_gender_dif',
           'dry_educ_alllevels_w_privschool_dif','msn_educ_alllevels_w_privschool_dif', 'winter_educ_alllevels_w_privschool_dif',
           'dry_educ_gender_dif_hrs', 'msn_educ_gender_dif_hrs','winter_educ_gender_dif_hrs',
           'dry_educ_allgender_dif_hrs','msn_educ_allgender_dif_hrs', 'winter_educ_allgender_dif_hrs',
           'dry_health_idx', 'msn_health_idx', 'winter_health_idx',
           'dry_markets_idx', 'msn_markets_idx', 'winter_markets_idx',
           'dry_admin_idx', 'msn_admin_idx', 'winter_admin_idx',
           'educ_eqwt_idx','educ_altwt_idx', 'health_idx', 'markets_idx', 'admin_idx',
           'overall_eq_wt_rank', 'overall_eq_wt_idx', 'overall_educ_altwt_rank','overall_educ_altwt_idx', 'overall_educ_priority_wt_rank', 'overall_educ_priority_wt_idx'] + list(adm_cols))], adm3_geo[['Adm3_Code','geometry']],on='Adm3_Code').sort_values('overall_eq_wt_rank')

    # geopackages can't handle categorical variable types
    adm3_geo.Elevation_category = adm3_geo.Elevation_category.astype(str)
    
    # export
    gpd.GeoDataFrame(adm3_geo,geometry='geometry').to_file(os.path.join(data_dir,acc_dir,f"vector/adm3_idx_access_stats_{today}.gpkg"),driver="GPKG")
else:
    None